# Stacking

Несмотря на то, что в открытом доступе существует довольно много реализаций стекинга, некоторые из которых даже представлены в виде библиотечных функций, лучше сделать собственную. Стекинг - не классический алгоритм решения задачи, а скорее набор правил для помощи в решении задачи другим алгоритмам. Если вы серьезно займетесь машинным обучением, рано или поздно вам скорее всего захочется что-нибудь поменять в этом наборе правил, поэтому собственная реализация с понятным вам кодом будет как нельзя кстати. 
Создадим свою функцию стеккинга:

In [3]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [4]:
def stacking(models, meta_alg, data_train, targets_train, data_test, targets_test=None, random_state=None, test_size=None, cv=5):
    
    
    ### функция стеккинга, принимающая на вход: 
    ### models - список моделей,
    ### meta_alg - модель на основе которой будет выдаваться предикт по стеккингу
    ### data_train - тренировочные данные
    ### targets_train - тренировочные целевые признаки
    ### data_test - тестовые данные
    ### targets_test - тестовые целевые признаки (по умалчанию None)
    ### random_state - по умолчанию None
    ### test_size - размер тестовой выборки ( по умолчанию None)
    
    
    if test_size is None:                                           # если размер выборки не задан - сделаем predict
                                                                    # с помощью  cross_val_predict
        meta_mtrx = np.empty((data_train.shape[0], len(models)))    # определим матрицу мета признаков
        
        for n, model in enumerate(models):                          # заполним матрицу признаков значениями cross_val_predict
            meta_mtrx[:, n] = cross_val_predict(model, data_train, targets_train, cv=cv, method='predict')
            model.fit(data_train, targets_train)                    # обучим наши модели
        meta = meta_alg
        meta_model = meta.fit(meta_mtrx, targets_train)             # обучим мета-алгоритм на матрице признаков
        
        meta_mtrx_test = np.empty((data_test.shape[0], len(models)))#определим матрицу мета признаков
        
        for n, model in enumerate(models):
            meta_mtrx_test[:, n] = model.predict(data_test)         # заполним матрицу предиктами на тестовых данных
    
        meta_predict = meta_model.predict(meta_mtrx_test)           # cделаем предикт для матрицы meta_mtrx_test

        if targets_test != None:                                    # выведем roc_auc_score
            roc_auc = roc_auc_score(targets_test, meta_predict)
            print(f'Stacking AUC: {roc_auc_score(targets_test, meta_predict)}')
            
        return meta_predict
    
    elif test_size > 0 and test_size < 1:                           # если размер выборки задан - реализуем свой стекинг
        train, valid, train_true, valid_true = train_test_split(data_train, 
                                                        targets_train,
                                                        test_size=test_size,
                                                        random_state=17)
                                                                    # зададим train и valid выборки
            
        meta_mtrx = np.empty((valid.shape[0], len(models)))         #определим матрицу мета признаков
                                                                    
            
        for n, model in enumerate(models):                          # заполним матрицу признаков значениями predict
            model.fit(train, train_true)                            # наших моделей
            meta_mtrx[:, n] = model.predict(valid)
        
        meta_model = meta_alg.fit(meta_mtrx, valid_true)            # обучим мета-алгоритм на нашей матрице мета признаков
        
        meta_mtrx_test = np.empty((data_test.shape[0], len(models)))# определим матрицу для мета признаков при обучении
        for n, model in enumerate(models):                          # на тестовых данных
            meta_mtrx_test[:, n] = model.predict(data_test)
        
        meta_predict = meta_model.predict(meta_mtrx_test)           # сделаем предикт для матрицы meta_mtrx_test
        
        if targets_test != None:                                    # выведем roc_auc_score
            roc_auc = roc_auc_score(targets_test, meta_predict)
            print(f'Stacking AUC: {roc_auc_score(targets_test, meta_predict)}')
            
        return meta_predict
    
    else:                                                           # вернём ошибку, если размер выборки находится
                                                                    # за перделами 0 - 1 
        raise ValueError("test_size must be between 0 and 1")

Базовая функция стекинга готова. Теперь проверим, как она работает. Ниже представлен датасет Titanic, разделенный на тренировочный и тестовый датасеты; предопределенные базовые алгоритмы и мета-алгоритм. Cоставим список базовых алгоритмов и запустим функцию в трех различных вариантах (при этом в каждом из них все значения data_train, targets_train, data_test, targets_test должны быть определены):

1. Вызвать исключение "test_size must be between 0 and 1".

2. Установить test_size=0.3; вывести AUC и массив полученных предсказаний.

3. Оставить test_size=None; вывести AUC и массив полученных предсказаний.

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.model_selection import cross_val_predict


titanic = pd.read_csv('./9.7_titanic.csv')
targets = titanic.Survived
data = titanic.drop(columns='Survived')

x_train, x_test, y_train, y_test = train_test_split(data, 
                                                    targets,
                                                    train_size=0.8,
                                                    random_state=17)

knn = KNeighborsClassifier(n_neighbors=3)
lr = LogisticRegression(random_state=17)
svc = SVC(random_state=17)

models = [knn, lr, svc]
meta_alg = XGBClassifier(n_estimators=40)

##### 1.Вызвать исключение "test_size must be between 0 and 1".

In [6]:
stacking(models=models,
         meta_alg=meta_alg,
         data_train=x_train,
         targets_train=y_train,
         data_test=x_test,
         test_size=5)

ValueError: test_size must be between 0 and 1

##### 2.Установить test_size=0.3; вывести AUC и массив полученных предсказаний.

In [7]:
meta_predict = stacking(models=models,
         meta_alg=meta_alg,
         data_train=x_train,
         targets_train=y_train,
         data_test=x_test,
         test_size=.3)

In [8]:
print(f'Stacking AUC: {roc_auc_score(y_test.values, meta_predict)}')

Stacking AUC: 0.7403721891961748


##### 3.Оставить test_size=None; вывести AUC и массив полученных предсказаний.

In [9]:
meta_predict = stacking(models=models,
         meta_alg=meta_alg,
         data_train=x_train,
         targets_train=y_train,
         data_test=x_test)

In [10]:
print(f'Stacking AUC: {roc_auc_score(y_test.values, meta_predict)}')

Stacking AUC: 0.7601447402429569
